In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import folium
from branca.element import Figure
from folium.plugins import HeatMapWithTime
from folium import plugins

In [2]:
geo = pd.read_csv('sdot_2016_2019.csv')
geo['INCDATE'] = pd.to_datetime(geo['INCDATE'])
geo['INCDTTM'] = pd.to_datetime(geo['INCDTTM'])
geo['hour'] = geo['INCDTTM'].dt.hour
pd.options.display.max_columns = None
display(geo)

,Unnamed: 0,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,year,dayofweek,week_number,month,weekend,LocNaN,hour
0,0,-122.386772,47.564720,1,326234,327734,E984735,Matched,Intersection,31893.0,CALIFORNIA AVE SW AND SW GENESEE ST,,NaN,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2019-11-20 00:00:00+00:00,2019-11-20 10:25:00,At Intersection (intersection related),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,N,Clear,Dry,Daylight,Y,NaN,NaN,2,Vehicle turning left hits pedestrian,0,0,N,2019,2,47,11,0,False,10
1,1,-122.341806,47.686934,2,326246,327746,E985430,Matched,Intersection,24228.0,STONE AVE N AND N 80TH ST,,NaN,2,Injury Collision,Angles,4,0,0,2,2,0,0,2019-11-18 00:00:00+00:00,2019-11-18 17:01:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Raining,Wet,Dark - Street Lights On,NaN,NaN,NaN,10,Entering at angle,0,0,N,2019,0,47,11,0,False,17
2,6,-122.323569,47.722577,7,320602,322102,3796466,Matched,Block,NaN,5TH AVE NE BETWEEN 130TH ST OFF RP AND NE 130T...,,NaN,1,Property Damage Only Collision,Other,2,0,0,2,0,0,0,2019-06-16 00:00:00+00:00,2019-06-16 01:07:00,Mid-Block (not related to intersection),28.0,MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT,NaN,N,Clear,Dry,Dark - Street Lights On,NaN,NaN,NaN,50,Fixed object,0,0,N,2019,6,24,6,1,False,1
3,8,-122.328878,47.733182,9,326166,327666,3751600,Matched,Block,NaN,1ST AVE NE BETWEEN N 143RD ST AND NE 145TH ST,,NaN,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2019-11-21 00:00:00+00:00,2019-11-21 18:08:00,Mid-Block (not related to intersection),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,N,Clear,Dry,Dark - Street Lights On,Y,NaN,NaN,0,Vehicle going straight hits pedestrian,0,521184,N,2019,3,47,11,0,False,18
4,12,-122.332365,47.665028,13,320518,322018,3646216,Matched,Block,NaN,N 50TH ST BETWEEN MERIDIAN AVE N AND KEYSTONE ...,,NaN,1,Property Damage Only Collision,Rear Ended,4,0,0,2,0,0,0,2019-06-21 00:00:00+00:00,2019-06-21 21:45:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Clear,Dry,Dark - Street Lights On,NaN,NaN,NaN,14,From same direction - both going straight - on...,0,0,N,2019,4,25,6,1,False,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49829,221261,-122.329060,47.605233,221262,318579,320079,E909186,Matched,Block,NaN,6TH AVE BETWEEN CHERRY ST AND COLUMBIA ST,,NaN,1,Property Damage Only Collision,Rear Ended,2,0,0,2,0,0,0,2019-03-07 00:00:00+00:00,2019-03-07 14:00:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Unknown,Unknown,Daylight,NaN,NaN,NaN,14,From same direction - both going straight - on...,0,0,N,2019,3,10,3,0,False,14
49830,221262,-122.314961,47.523468,221263,318368,319868,3780990,Matched,Block,NaN,14TH AVE S BETWEEN S CONCORD ST AND S HENDERSO...,,NaN,2,Injury Collision,Rear Ended,2,0,0,2,1,0,0,2019-04-26 00:00:00+00:00,2019-04-26 09:59:00,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,14,From same direction - both going straight - on...,0,0,N,2019,4,17,4,1,False,9
49831,221263,-122.334203,47.575576,221264,318948,320448,E920056,Matched,Intersection,31379.0,1ST AVE S AND S HANFORD ST,,NaN,1,Property Damage Only Collision,Right Turn,2,0,0,2,0,0,0,2019-05-09 00:00:00+00:00,2019-05-09 07:00:00,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,16,From sa

In [3]:
geo.shape

(49834, 48)

In [4]:
sdot = pd.read_csv('sdot_2016_2019.csv')
sdot['LocNaN'] = pd.Series(dtype='object')
sdot['LocNaN'] = np.where(sdot.Y.isna(), 'True', 'False')
geo_s = geo[geo.LocNaN !=True]  #Removing rows with Y or X NaN


In [5]:
geo_s.shape   #Confirming X or Y NaN rows removed

(48292, 48)

In [6]:
#drop unneeded columns
geo_s = geo_s[['X','Y','INCKEY','SEVERITYDESC','INCDTTM','hour','weekend','WEATHER','ROADCOND','LIGHTCOND','ADDRTYPE','JUNCTIONTYPE']]
geo_s.head()

,X,Y,INCKEY,SEVERITYDESC,INCDTTM,hour,weekend,WEATHER,ROADCOND,LIGHTCOND,ADDRTYPE,JUNCTIONTYPE
0,-122.386772,47.564720,326234,Injury Collision,2019-11-20 10:25:00,10,0,Clear,Dry,Daylight,Intersection,At Intersection (intersection related)
1,-122.341806,47.686934,326246,Injury Collision,2019-11-18 17:01:00,17,0,Raining,Wet,Dark - Street Lights On,Intersection,At Intersection (intersection related)
2,-122.323569,47.722577,320602,Property Damage Only Collision,2019-06-16 01:07:00,1,1,Clear,Dry,Dark - Street Lights On,Block,Mid-Block (not related to intersection)
3,-122.328878,47.733182,326166,Injury Collision,2019-11-21 18:08:00,18,0,Clear,Dry,Dark - Street Lights On,Block,Mid-Block (not related to intersection)
4,-122.332365,47.665028,320518,Property Damage Only Collision,2019-06-21 21:45:00,21,1,Clear,Dry,Dark - Street Lights On,Block,Mid-Block (not related to intersection)


In [7]:
geo_s.duplicated().value_counts()  #checking if duplicates
geo_s=geo_s.drop_duplicates()      #removing duplicates
geo_s.isnull().sum()               #checking if any null

X                  0
Y                  0
INCKEY             0
SEVERITYDESC       0
INCDTTM            0
hour               0
weekend            0
WEATHER         8381
ROADCOND        8332
LIGHTCOND       8408
ADDRTYPE           0
JUNCTIONTYPE    2680
dtype: int64

In [8]:
#We'll look at weekdays and weekends separately
geo_s_WD = geo_s[geo_s.weekend == 0]  #Weekdays
geo_s_WE = geo_s[geo_s.weekend == 1]  #Weekends
print("There are ", geo_s_WD.shape, "weekday records and ", geo_s_WE.shape, "weekend records.")

There are  (28823, 12) weekday records and  (19469, 12) weekend records.


In [9]:
#Weekdays: Let’s convert the position of vehicles into this format:

lat_long_listD = []
for i in range(1,25):
    temp=[]
    for index, instance in geo_s_WD[geo_s_WD['hour'] == i].iterrows():
        temp.append([instance['Y'],instance['X']])
    lat_long_listD.append(temp)



In [10]:
!pip install folium --upgrade

Requirement already up-to-date: folium in /opt/anaconda3/lib/python3.8/site-packages (0.11.0)


In [11]:
from branca.element import CssLink, Element, Figure, JavascriptLink
from folium.plugins import HeatMapWithTime

mapD = folium.Map(location=[47.60621, -122.33207], zoom_start=12)   #Base map

fig_D=Figure(width=850,height=850)
fig_D.add_child(mapD)

fig_D.header.add_child(
    JavascriptLink('https://rawcdn.githack.com/nezasa/iso8601-js-period/master/iso8601.min.js'),
    name='iso8601')

plugins.HeatMapWithTime(lat_long_listD,radius=8,auto_play=True,position='bottomright').add_to(mapD)

mapD


In [12]:
#Weekends: Let’s convert the position of vehicles into this format:

lat_long_listE = []
for i in range(1,25):
    temp=[]
    for index, instance in geo_s_WE[geo_s_WE['hour'] == i].iterrows():
        temp.append([instance['Y'],instance['X']])
    lat_long_listE.append(temp)

mapE = folium.Map(location=[47.60621, -122.33207], zoom_start=12)   

fig_E=Figure(width=850,height=850)
fig_E.add_child(mapE)

fig_E.header.add_child(
    JavascriptLink('https://rawcdn.githack.com/nezasa/iso8601-js-period/master/iso8601.min.js'),
    name='iso8601')

plugins.HeatMapWithTime(lat_long_listD,radius=8,auto_play=True,position='bottomright').add_to(mapE)

mapE

In [13]:
#Injuries only
geos_INJ = geo_s[(geo_s.SEVERITYDESC == 'Injury Collision') | (geo_s.SEVERITYDESC == 'Serious Injury Collision') | (geo_s.SEVERITYDESC == 'Fatality Collision')] 

lat_long_list_INJ = []

for i in range(1,25):
    temp=[]
    for index, instance in geos_INJ[geos_INJ['hour'] == i].iterrows():
        temp.append([instance['Y'],instance['X']])
    lat_long_list_INJ.append(temp)

mapINJ = folium.Map(location=[47.60621, -122.33207], zoom_start=12)   

fig_INJ=Figure(width=850,height=850)
fig_INJ.add_child(mapINJ)

fig_INJ.header.add_child(
    JavascriptLink('https://rawcdn.githack.com/nezasa/iso8601-js-period/master/iso8601.min.js'),
    name='iso8601')

plugins.HeatMapWithTime(lat_long_list_INJ,radius=8,auto_play=True,position='bottomleft').add_to(mapINJ)

mapINJ


In [14]:
geo_s['LIGHTCOND'].value_counts()

Daylight                    25945
Dark - Street Lights On     10319
Dusk                         1260
Unknown                      1030
Dawn                          662
Dark - No Street Lights       368
Dark - Street Lights Off      242
Other                          58
Name: LIGHTCOND, dtype: int64

In [15]:
#Create a map by LightCond

coords_1 = (geo_s[geo_s.LIGHTCOND == 'Daylight']).sort_values(by=['Y','X'])
coords_2 = (geo_s[geo_s.LIGHTCOND == 'Dark - Street Lights On']).sort_values(by=['Y','X'])
coords_3 = (geo_s[geo_s.LIGHTCOND == 'Dusk']).sort_values(by=['Y','X'])
coords_4 = (geo_s[geo_s.LIGHTCOND == 'Dawn']).sort_values(by=['Y','X'])
coords_5 = (geo_s[geo_s.LIGHTCOND == 'Dark - No Street Lights']).sort_values(by=['Y','X'])
coords_6 = (geo_s[geo_s.LIGHTCOND == 'Dark - Street Lights Off']).sort_values(by=['Y','X'])
coords_7 = (geo_s[(geo_s.LIGHTCOND == 'Unknown') | (geo_s.LIGHTCOND == 'Other')]).sort_values(by=['Y','X'])

cds_1 = (coords_1[['Y','X']]).to_numpy()
cds_2 = (coords_2[['Y','X']]).to_numpy()
cds_3 = (coords_3[['Y','X']]).to_numpy()
cds_4 = (coords_4[['Y','X']]).to_numpy()
cds_5 = (coords_5[['Y','X']]).to_numpy()
cds_6 = (coords_6[['Y','X']]).to_numpy()
cds_7 = (coords_7[['Y','X']]).to_numpy()

cds_7

array([[  47.49648571, -122.26428944],
       [  47.49778198, -122.26277242],
       [  47.50053325, -122.37729376],
       ...,
       [  47.73379124, -122.2971996 ],
       [  47.73409487, -122.32887984],
       [  47.73413054, -122.34239191]])

In [16]:
mapLC = folium.Map(location=[47.60621, -122.33207], tiles='cartodbpositron', zoom_start=12)  

figLC=Figure(height=550,width=750)
figLC.add_child(mapLC)

# Creating feature groups
f1=folium.FeatureGroup('Daylight')
f2=folium.FeatureGroup('Dark - Street Lights On')
f3=folium.FeatureGroup('Dusk')
f4=folium.FeatureGroup('Dawn')
f5=folium.FeatureGroup('Dark - No Street Lights')
f6=folium.FeatureGroup('Dark - Street Lights Off')
f7=folium.FeatureGroup('Unknown or Other')

# Adding lines to the different feature groups

line_1=folium.PolyLine(coords_1.Y, coords_1.X, popup='<b>Daylight</b>', tooltip='Daylight', color='blue', weight=10).add_to(f1)
line_2=folium.PolyLine([coords_2['Y']],[coords_2['X']],popup='<b>Dark - Street Lights On</b>',tooltip='Dark - Street Lights On',color='red',weight=10).add_to(f2)
line_3=folium.PolyLine([coords_3['Y']],[coords_3['X']],popup='<b>Dusk</b>',tooltip='Dusk',color='green',weight=10).add_to(f3)
line_4=folium.PolyLine([coords_4['Y']],[coords_4['X']],popup='<b>Dawn</b>',tooltip='Dawn',color='orange',weight=10).add_to(f4)
line_5=folium.PolyLine([coords_5['Y']],[coords_5['X']],popup='<b>Dark - No Street Lights</b>',tooltip='Dark - No Street Lights',color='purple',weight=10).add_to(f5)
line_6=folium.PolyLine([coords_6['Y']],[coords_6['X']],popup='<b>Dark - Street Lights Off</b>',tooltip='Dark - Street Lights Off',color='brown',weight=10).add_to(f6)
line_7=folium.PolyLine([coords_7['Y']],[coords_7['X']],popup='<b>Unknown or Other</b>',tooltip='Unknown or Other',color='gray',weight=10).add_to(f7)

f1.add_to(mapLC)
f2.add_to(mapLC)
f3.add_to(mapLC)
f4.add_to(mapLC)
f5.add_to(mapLC)
f6.add_to(mapLC)
f7.add_to(mapLC)
folium.LayerControl().add_to(mapLC)
mapLC

TypeError: __init__() got multiple values for argument 'popup'